## 処理の高速化をする４つの方法
<hr>
動くプログラムを作成した後に処理の高速化を考える際、参考になる
４つの考え方を紹介します。

- 組み込み関数の使用

- 外部モジュールの使用

- アルゴリズムの最適化


実行環境<br>
[python3.10.10](https://docs.python.org/3.10/)<br>
[Numpy1.24.2](https://numpy.org/)


## 組み込み関数を使おう
<hr>

<span style="font-size:75%">1から100000000までの数の総和を計算する。</span><br>
自作コードの使用
```python
import time
start_time = time.time()
total = 0
for i in range(1,100000001):
    total += i
end_time = time.time()
print("計算結果:",total,"\nsumの処理時間: ", end_time - start_time)
#forループの処理時間: 8.668177366256714
```

組み込み関数（sum()）の使用
```python
import time
start_time = time.time()
total = sum(list(range(1,100000001)))
end_time = time.time()
print("計算結果:",total,"\nsumの処理時間: ", end_time - start_time)
#forループの処理時間:4.457662105560303
```

## 外部モジュールを使おう
<hr>
プログラミング言語により処理速度が異なり、<br>
pythonは処理速度が遅い。<br>
外部モジュールを用いることで早い言語で処理を行える。


Numpyを使用
```python
import time
start_time = time.time()
import numpy
total = numpy.sum(numpy.arange(1,100000001))
end_time = time.time()
print("計算結果:",total,"\nnumpyの処理時間: ", end_time - start_time)
#forループの処理時間:0.5025210380554199
```
実用例：RでCNSとSVの重なりを[biocoductor](https://www.bioconductor.org/)で検出

## アルゴリズムを見直そう
<hr>
アルゴリズムを見直す際以下の視点がある。<br>
- 同じ処理をまとめる<br>
- データの読み書きを減らす<br>
- 型の変換を減らす<br>

総和の公式($\frac n2(1+n)$)を使用
```python
start_time = time.time()
total = int(100000000*(1+100000001)/2)
end_time = time.time()
print("計算結果:",total,"\nアルゴリズムの変更後の処理時間: ", end_time - start_time)
#forループの処理時間:1.1920928955078125e-06
```

## 注意点

- 高速化後は結果が変わっていないかを確認しよう<br>
計算過程に少数が入ると整数への変換時に結果が変わることがある
```python
print(3/2 + 2/3) # 2.16666
print(int(2/3) + int(3/2)) #1
```
<br>


- 適切な方法を使おう<br>
処理によっては紹介した方法では遅くなる場合もある<br>
１行づつ実行し遅い箇所を特定し最適な方法を探そう

## おまけ1：並列化をしよう
<hr>
複数の処理を同時におこなったり、処理を分割して<br>並列処理することで速度が上がることがある。


In [ ]:
#| eval: false
import time
start_time = time.time()
from multiprocessing import Process, Value, Lock
def cal(start, end, sum_cal, lock):
    local_sum = 0
    for i in range(start, end+1):
        local_sum += i
    with lock:
        sum_cal.value +=local_sum
if __name__ == '__main__':
    n =  100000000
    sum_cal = Value('l', 0)
    lock = Lock()
    half = int(n/2)
    p1 = Process(target=cal, args=(1, half, sum_cal, lock))
    p2 = Process(target=cal, args=(half+1, n, sum_cal, lock))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    total_sum = sum_cal.value
    end_time = time.time()
    print(total_sum,end_time - start_time)

## おまけ2：言語を見直そう
<hr>

コンパイル型言語の使用によりインタプリンタ型言語の<br>
高速化と同じ結果が望める。

```C++
#include <iostream>
#include <chrono>
using namespace std;
int main()
{   int n = 100000000;
    long sum_ = 0;
    auto start = chrono::steady_clock::now(); // 開始時刻
    for (int i = 1; i <= n; i++) {
        sum_ += i;}
    auto end = chrono::steady_clock::now(); // 終了時刻
    auto diff = end - start; // 経過時間
    cout << "Sum = " << sum_ << endl;
    cout << "Elapsed time = " << chrono::duration<double, milli>(diff).count() << " ms" << endl;
    return 0;}
/*C++でのfor文処理時間:0.224624*/
```
<span style="font-size:50%">[chatGPT](https://openai.com/blog/chatgpt)により生成したものを修正</span>
